In [ ]:
include("utils.jl")
using PyCall, PyPlot, LaTeXStrings

rcParams = PyDict(matplotlib["rcParams"])
rcParams["xtick.direction"] = "in"
rcParams["ytick.direction"] = "in"
rcParams["font.size"] = 8
rcParams["text.usetex"] = true
rcParams["figure.dpi"] = 300

mkpath("figures")

In [ ]:
fig, axs = subplots(3, 1, figsize=(2.0, 3.0), layout="constrained")

key = "3D-LJ-T1.5"
datadir = "data/$(key)"
sim_ids = [15, 13, 10]

for (sim_id, ax) in zip(sim_ids, axs)
    ax2 = ax.twinx()
    ax.set_zorder(1)
    ax.set_frame_on(false)
    file = jldopen(readdir(datadir, join=true)[sim_id], "r")
    Vext = file["Vext"]
    replace!(x -> !isfinite(x) ? 100 : x, Vext)
    ax2.plot(file["xs"], Vext, color="tab:gray", linewidth=1, alpha=0.5)
    ax2.fill_between(file["xs"], Vext, -100, color="lightgray", alpha=0.1)
    ax2.set_ylim(-10, 10)
    ax2.set_yticks([-10, -5, 0, 5, 10])
    ax2.set_ylabel(L"V_\mathrm{ext} / \varepsilon", color="gray", labelpad=-3)
    ax2.tick_params(axis="y", colors="gray")
    
    ax.plot(file["xs"], file["ρ"], color="tab:blue")
    ax.set_ylabel(L"\rho \sigma^3", labelpad=2)
    ax.set_ylim(0, 2.5)
    ax.set_xlim(0, 20)
    ax.set_xticks([0, 20])
    ax.set_xlabel(L"x / \sigma", labelpad=-8)
end

fig.show()
fig.savefig("figures/training_systems.pdf")

In [ ]:
systems = Dict(
    "1D-hard-rods" => Dict(
        "title" => "hard rods",
        "subtitle" => "",
        "c1_with_T" => false,
        "plot_prediction" => ax -> begin
            file = jldopen("predictions/walls_1D-hard-rods.jld2")
            μ = file["μ"]
            xs = file["xs"] .- 2.5
            ax.text(0.5, 0.07, L"\beta \mu = %$μ", transform=ax.transAxes, ha="center")
            ax.plot(xs, file["ρ_neural"])
            ax.plot(xs, file["ρ_Percus"], linestyle="dotted", linewidth=1, color="black", label="Percus")
            ax.legend(loc="upper center")
        end
    ),
    "3D-hard-spheres" => Dict(
        "title" => "hard spheres",
        "subtitle" => "",
        "c1_with_T" => false,
        "plot_prediction" => ax -> begin
            file = jldopen("predictions/walls_3D-hard-spheres.jld2")
            sim = jldopen("predictions/walls_3D-hard-spheres_sim.hdf5")
            μ = file["μ"]
            xs = file["xs"] .- 2.5
            ax.text(0.5, 0.07, L"\beta \mu = %$μ", transform=ax.transAxes, ha="center")
            ax.plot(xs, file["ρ_neural"])
            ax.plot(xs, file["ρ_neural_only_c1"], linestyle="dashed", linewidth=1, color="orange", label=L"c_1" * "-training")
            ax.plot(sim["xs"] .- 2.5, sim["ρ"], linestyle="dotted", linewidth=1, color="black", label="simulation")
            ax.legend(loc="upper center")
        end
    ),
    "3D-LJ-T1.5" => Dict(
        "title" => "Lennard-Jones",
        "subtitle" => L"k_B T = 1.5 \varepsilon",
        "c1_with_T" => false,
        "plot_prediction" => ax -> begin
            file = jldopen("predictions/walls_3D-LJ-T1.5.jld2")
            sim = jldopen("predictions/walls_3D-LJ-T1.5_sim.hdf5")
            μ = file["μ"]
            xs = file["xs"] .- 2.5
            ax.text(0.5, 0.07, L"\beta \mu = %$μ", transform=ax.transAxes, ha="center")
            ax.plot(xs, file["ρ_neural"])
            ax.plot(xs, file["ρ_neural_only_c1"], linestyle="dashed", linewidth=1, color="orange", label=L"c_1" * "-training")
            ax.plot(sim["xs"] .- 2.5, sim["ρ"], linestyle="dotted", linewidth=1, color="black", label="simulation")
            ax.legend(loc="upper center")
        end
    ),
    "3D-LJ-Tvar" => Dict(
        "title" => "Lennard-Jones",
        "subtitle" => L"1.0 < k_B T / \varepsilon < 2.0",
        "c1_with_T" => true,
        "plot_prediction" => ax -> begin
            function get_dividing_surface(xs, ρ)
                ρl = ρ[1]
                ρg = ρ[length(xs) ÷ 2]
                xs[ρ .< (ρl + ρg) / 2][1]
            end
            file = jldopen("predictions/coex_3D-LJ-Tvar.jld2")
            prx = jldopen("predictions/coex_3D-LJ-Tvar_PRX.hdf5")
            xs = file["xs"]
            ρ = file["ρ_neural"]
            dividing_surface = get_dividing_surface(xs, ρ)
            xs_prx = prx["xs"]
            ρ_prx = prx["ρ_neural"][:,prx["Ts"].≈file["T"]]
            dividing_surface_prx = get_dividing_surface(xs_prx, ρ_prx)
            ax.text(0.05, 0.07, L"k_B T / \varepsilon = 1.0", transform=ax.transAxes, ha="left")
            ax.plot(xs .- dividing_surface, ρ)
            ax.plot(xs_prx .- dividing_surface_prx, ρ_prx, label="Ref.~[61]", linestyle="dotted", color="black", linewidth=1)
            ax.set_xlim(-5, 5)
            ax.set_ylim(0, 0.8)
            ax.legend(loc="upper right")
        end
    ),
)

systems_plot = ["1D-hard-rods", "3D-LJ-Tvar"]
#systems_plot = ["3D-hard-spheres", "3D-LJ-T1.5"]

fig, axs = subplots(2, 2, figsize=(3.37, 3.8), height_ratios=[1.5, 1], layout="constrained")
axsins = [ax[1].inset_axes([0, -0.37, 1.0, 0.3], transform=ax[1].transAxes) for ax in eachcol(axs)]
labels = ["(a)", "(b)"]
axs[1].set_ylabel(L"\beta \mu_\mathrm{fit}")
axs[2].set_ylabel(L"\rho \sigma^d")
axsins[1].set_ylabel(L"\beta \Delta \mu", labelpad=-3)

for (key, ax, axins, label) in zip(systems_plot, eachcol(axs), axsins, labels)
    ax[1].text(0.5, 1.15, systems[key]["title"], transform=ax[1].transAxes, ha="center", fontsize=9)
    ax[1].text(0.5, 1.05, systems[key]["subtitle"], transform=ax[1].transAxes, ha="center")
    ax[1].text(0., 1.15, label, transform=ax[1].transAxes)
    datadir = "data/$(key)"
    ρ_profiles, Vext_profiles, μ_values, T_values = read_sim_data(datadir)
    model_state_history = JLD2.load("models/model_state_history_$(key).jld2", "model_state_history")
    window_bins, c1_with_T = get_model_input(model_state_history[end])
    model = load_trained_model("models/model_state_history_$(key).jld2")
    c1_ρ0 = model.c1(zeros(Float32, window_bins, length(T_values)), reshape(T_values, 1, :)) |> vec
    μ_lims = [minimum(μ_values ./ T_values)-1, maximum(μ_values ./ T_values)+1]
    ax[1].plot(μ_values ./ T_values, (model.μ.weight[1,:] .+ c1_ρ0 .* T_values) ./ T_values, ".", alpha=0.1, color="tab:purple")
    ax[1].set_xticks([-10, -5, 0, 5, 10])
    ax[1].set_xticklabels([])
    ax[1].set_yticks([-10, -5, 0, 5, 10])
    ax[1].set_xlim(μ_lims)
    ax[1].set_ylim(μ_lims)
    ax[1].plot(μ_lims, μ_lims, color="black", linestyle="dashed", linewidth=0.5)
    ax[1].set_aspect("equal")
    axins.plot(μ_values ./ T_values, (model.μ.weight[1,:] .+ c1_ρ0 .* T_values .- μ_values) ./ T_values, ".", alpha=0.1, color="tab:purple")
    axins.plot(μ_lims, zero(μ_lims), color="black", linestyle="dashed", linewidth=0.5)
    axins.set_xticks([-10, -5, 0, 5, 10])
    axins.set_xlim(μ_lims)
    axins.set_ylim(-0.1, 0.1)
    axins.set_xlabel(L"\beta \mu_\mathrm{sim}", labelpad=0)
    ax[2].set_xlabel(L"x / \sigma")
    ax[2].set_xlim(-1, 11)
    ax[2].set_ylim(0, 1.7)
    systems[key]["plot_prediction"](ax[2])
end

fig.show()
fig.savefig("figures/$(join(systems_plot, '_')).pdf")

In [ ]:
for key in ["1D-hard-rods", "3D-hard-spheres", "3D-LJ-T1.5", "3D-LJ-Tvar"]
    datadir = "data/$(key)"
    ρ_profiles, Vext_profiles, μ_values, T_values = read_sim_data(datadir)
    model_state_history = JLD2.load("models/model_state_history_$(key).jld2", "model_state_history")
    window_bins, c1_with_T = get_model_input(model_state_history[end])
    model = load_trained_model("models/model_state_history_$(key).jld2")
    model = model |> cpu
    μ_lims = [minimum(μ_values)-1, maximum(μ_values)+1]
    T_lims = [minimum(T_values)-0.1, maximum(T_values)+0.1]
    a = Animation()
    a_offset = Animation()
    for (i, model_state) in enumerate(model_state_history)
        Flux.loadmodel!(model, model_state)
        c1_ρ0 = model.c1(zeros(Float32, window_bins, length(T_values)), reshape(T_values, 1, :)) |> vec
        pμ = Plots.scatter(μ_values, model.μ.weight[1,:] .+ c1_ρ0 .* model.T.weight[1,:]; aspect_ratio=1, xlim=μ_lims, ylim=μ_lims, legend=false, xlabel="μ_sim", ylabel="μ_fit")
        plot!(pμ, μ_lims, μ_lims; linestyle=:dash, color="black")
        pT = Plots.scatter(T_values, model.T.weight[1,:]; aspect_ratio=1, xlim=T_lims, ylim=T_lims, legend=false, xlabel="T_sim", ylabel="T_fit")
        plot!(pT, T_lims, T_lims; linestyle=:dash, color="black")
        p = Plots.plot(pμ, pT; layout=(2, 1), size=(300, 600), plot_title="epoch $(i-1)")
        frame(a, p)
        p_offset = Plots.scatter(T_values, c1_ρ0 .* model.T.weight[1,:]; legend=false, xlim=T_lims, ylim=(-15, 15), xlabel="T_sim", ylabel="offset", plot_title="epoch $(i-1)")
        frame(a_offset, p_offset)
    end
    display(gif(a, "figures/animations/mu_T_$(key).gif", fps=10))
    display(gif(a_offset, "figures/animations/offset_$(key).gif", fps=10))
end

In [ ]:
fig, ax = subplots(1, 1, figsize=(3.0, 2.3), layout="constrained")

file = jldopen("predictions/walls_3D-LJ-T1.5.jld2")
sim = jldopen("predictions/walls_3D-LJ-T1.5_sim.hdf5")
μ = file["μ"]
xs = file["xs"] .- 2.5
ax.text(0.5, 1.12, "Lennard-Jones", transform=ax.transAxes, ha="center", fontsize=9)
ax.text(0.5, 1.04, L"k_B T = 1.5 \varepsilon", transform=ax.transAxes, ha="center")
ax.text(0.5, 0.07, L"\beta \mu = %$μ", transform=ax.transAxes, ha="center")
ax.plot(xs, file["ρ_neural_canonical"], label="canonical training")
ax.plot(xs, file["ρ_neural"], linestyle="dashed", linewidth=1, color="orange", label="grand canonical training")
ax.plot(sim["xs"] .- 2.5, sim["ρ"], linestyle="dotted", linewidth=1, color="black", label="grand canonical simulation")
ax.set_xlim(-1, 11)
ax.set_ylim(0, 1.7)
ax.set_yticks([0, 0.5, 1, 1.5])
ax.set_xlabel(L"x / \sigma")
ax.set_ylabel(L"\rho \sigma^3")
ax.legend(loc="upper center")

fig.show()
fig.savefig("figures/canonical.pdf")

In [ ]:
fig, ax = subplots(1, 1, figsize=(2.5, 2.5), layout="constrained")

key = "3D-LJ-T1.5"
datadir = "data/$(key)"
ax.text(0.5, 1.12, "Lennard-Jones", transform=ax.transAxes, ha="center", fontsize=9)
ax.text(0.5, 1.04, L"k_B T = 1.5 \varepsilon", transform=ax.transAxes, ha="center")
ax.set_aspect("equal")
ax.set_xticks([-10, -5, 0, 5, 10])
ax.set_yticks([-10, -5, 0, 5, 10])
ax.set_xlabel(L"\beta \mu_\mathrm{sim}")
ax.set_ylabel(L"\beta \mu_\mathrm{fit}")
ρ_profiles, Vext_profiles, μ_values, T_values = read_sim_data(datadir)
μ_lims = [minimum(μ_values ./ T_values)-1, maximum(μ_values ./ T_values)+1]
ax.set_xlim(μ_lims)
ax.set_ylim(μ_lims)
ax.plot(μ_lims, μ_lims, color="black", linestyle="dashed", linewidth=0.5)
color = Dict(100 => "tab:pink", 50 => "tab:cyan", 20 => "tab:olive")
for n in [100, 50, 20]
    ρ_profiles, Vext_profiles, μ_values, T_values = read_sim_data(datadir; n_max=n)
    n_suffix = ""
    if isfinite(n)
        n_suffix = "_n$(n)"
    end
    model_state_history = JLD2.load("models/model_state_history_$(key)$(n_suffix).jld2", "model_state_history")
    window_bins, c1_with_T = get_model_input(model_state_history[end])
    model = load_trained_model("models/model_state_history_$(key)$(n_suffix).jld2")
    c1_ρ0 = model.c1(zeros(Float32, window_bins, length(T_values)), reshape(T_values, 1, :)) |> vec
    ax.plot(μ_values ./ T_values, (model.μ.weight[1,:] .+ c1_ρ0 .* T_values) ./ T_values, ".", alpha=0.5, color=color[n], label=L"n = %$n")
end
ax.legend()

fig.show()
fig.savefig("figures/reduce_size.pdf")